In [ ]:
!pip3 install --upgrade google-cloud-videointelligence
!pip3 install threading
!pip3 install google-cloud-speech

ERROR: Could not find a version that satisfies the requirement threading (from versions: none)
ERROR: No matching distribution found for threading


In [ ]:
from google.colab import drive

drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


Make a copy of 'Systems '22' at your own **MyDrive** 

In [ ]:
%ls gdrive/MyDrive/"Systems '22"/

'Capstone Presentation.pptx'
'Content Features Table (Labeled data - Xiao)'/
'Content Features Table (Unlabeled data) '/
 EDA/
 final_csv/
'Final Report.docx'
 github/
 google-cloud-sdk-377.0.0-linux-x86_64.tar.gz
'Labeled GCP Output (Xiao)'/
 labeled_video_content.gsheet
'Meeting Notes'/
'Metadata (Labeled data - Xiao) & Features Table'/
'Metadata (Unlabeled Data) & Features Table'/
'Project Course Guide.pdf'
'Raw videos(Xiao)'/
 Scripts/
'Summary (Ask Your Doctor to Prescribe a YouTube Video).docx'
 text/
 videoIDs.txt
 video_raw_content.csv
 videos/
 voice_gender_detection/
"YouTube Systems '22 - Team Bio Document.pdf"


In [ ]:
%cd gdrive/MyDrive/"Systems '22"/

/content/gdrive/.shortcut-targets-by-id/1ohwBGdC8wri16R_SjnGnOM_B619TDepK/Systems '22


In [ ]:
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

AuthorizationError: ignored

In [ ]:
BILLING_ACCOUNT_ID = '01074B-EAE851-71F16B'

GCP_PROJECT_ID = 'stoked-dominion-341317'
PROJECT_NUMBER = '725342657078'

!gcloud beta billing projects link $GCP_PROJECT_ID --billing-account $BILLING_ACCOUNT_ID

API [cloudbilling.googleapis.com] not enabled on project [522309567947]. Would 
you like to enable and retry (this will take a few minutes)? (y/N)?  y

Enabling service [cloudbilling.googleapis.com] on project [522309567947]...
ERROR: (gcloud.beta.billing.projects.link) PERMISSION_DENIED: Permission denied to enable service [cloudbilling.googleapis.com]
Help Token: Ae-hA1MQApT1vv9XEmb9N7QnfjQfHkWHTfTcZqrv5X47UwukZHLkjT7RXB9t0O-M-F3U0yAKslxpDtet9hBLDagM7cC9CycPBXQUvNGX8db9l_qM
- '@type': type.googleapis.com/google.rpc.PreconditionFailure
  violations:
  - subject: '110002'
    type: googleapis.com
- '@type': type.googleapis.com/google.rpc.ErrorInfo
  domain: serviceusage.googleapis.com
  reason: AUTH_PERMISSION_DENIED


In [ ]:
!gcloud beta billing accounts list

API [cloudbilling.googleapis.com] not enabled on project [522309567947]. Would 
you like to enable and retry (this will take a few minutes)? (y/N)?  y

Enabling service [cloudbilling.googleapis.com] on project [522309567947]...
ERROR: (gcloud.beta.billing.accounts.list) PERMISSION_DENIED: Permission denied to enable service [cloudbilling.googleapis.com]
Help Token: Ae-hA1NOekUqRDZjL4ETkfOdo5soAwvQApU2gmofx9QcUqto1m46Gq0VtoNm1bwQYivrK3iWgqPd-fkMoVIVGH4g4eOtA08JnJl9aKGkYVOtDnRg
- '@type': type.googleapis.com/google.rpc.PreconditionFailure
  violations:
  - subject: '110002'
    type: googleapis.com
- '@type': type.googleapis.com/google.rpc.ErrorInfo
  domain: serviceusage.googleapis.com
  reason: AUTH_PERMISSION_DENIED


In [ ]:
!gcloud services --project $GCP_PROJECT_ID enable ml.googleapis.com cloudbuild.googleapis.com

Operation "operations/acat.p2-725342657078-de2e6913-d279-4335-99bf-57e70a83d69e" finished successfully.


In [ ]:
SERVICE_ACCOUNT_NAME ='ytnotebook'

SERVICE_ACCOUNT_EMAIL = f'{SERVICE_ACCOUNT_NAME}@{GCP_PROJECT_ID}.iam.gserviceaccount.com'
#!gcloud iam --project $GCP_PROJECT_ID service-accounts create $SERVICE_ACCOUNT_NAME
!gcloud projects add-iam-policy-binding $GCP_PROJECT_ID \
    --member serviceAccount:$SERVICE_ACCOUNT_EMAIL \
    --role=roles/editor

Updated IAM policy for project [stoked-dominion-341317].
bindings:
- members:
  - user:ylingqin08@gmail.com
  role: roles/billing.projectManager
- members:
  - serviceAccount:725342657078@cloudbuild.gserviceaccount.com
  role: roles/cloudbuild.builds.builder
- members:
  - serviceAccount:service-725342657078@gcp-sa-cloudbuild.iam.gserviceaccount.com
  role: roles/cloudbuild.serviceAgent
- members:
  - serviceAccount:service-725342657078@gcf-admin-robot.iam.gserviceaccount.com
  role: roles/cloudfunctions.serviceAgent
- members:
  - serviceAccount:service-725342657078@cloudcomposer-accounts.iam.gserviceaccount.com
  role: roles/composer.serviceAgent
- members:
  - serviceAccount:service-725342657078@compute-system.iam.gserviceaccount.com
  role: roles/compute.serviceAgent
- members:
  - serviceAccount:service-725342657078@container-engine-robot.iam.gserviceaccount.com
  role: roles/container.serviceAgent
- members:
  - serviceAccount:service-725342657078@containerregistry.iam.gserviceac

In [ ]:
DEFAULT_AI_PLATFORM_SERVICE_ACCOUNT = f'service-{PROJECT_NUMBER}@cloud-ml.google.com.iam.gserviceaccount.com'

!gcloud iam --project $GCP_PROJECT_ID service-accounts add-iam-policy-binding \
--role=roles/iam.serviceAccountAdmin \
--member=serviceAccount:$DEFAULT_AI_PLATFORM_SERVICE_ACCOUNT \
$SERVICE_ACCOUNT_EMAIL

API [iam.googleapis.com] not enabled on project [522309567947]. Would you like 
to enable and retry (this will take a few minutes)? (y/N)?  y

Enabling service [iam.googleapis.com] on project [522309567947]...
ERROR: (gcloud.iam.service-accounts.add-iam-policy-binding) PERMISSION_DENIED: Permission denied to enable service [iam.googleapis.com]
Help Token: Ae-hA1PvQiK59O3-Qyk3Cx9XniARBuVtAGmr70AqN6zkAe-v3Z6we9Jcn6EMGimKAnQkWiWCQ0To50oAIBooRA9L5eYrgcqbJH8IdNFhjwb-kzTk
- '@type': type.googleapis.com/google.rpc.PreconditionFailure
  violations:
  - subject: '110002'
    type: googleapis.com
- '@type': type.googleapis.com/google.rpc.ErrorInfo
  domain: serviceusage.googleapis.com
  reason: AUTH_PERMISSION_DENIED


In [ ]:

import os, io
import pandas as pd
from google.cloud import videointelligence
from collections import namedtuple
import plotly.graph_objects as go
import plotly.express as px
import requests
import string
import proto
import json

## download your API Key from your Google Console

#this is the path to the API credentials. Modify this path to the json file in this folder.
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'Scripts/stoked-dominion-341317-24484c3fbb6f.json'
video_client = videointelligence.VideoIntelligenceServiceClient()


import difflib

In [ ]:
from google.cloud import storage


video_list = []
def list_blobs_with_prefix(bucket_name, prefix, delimiter=None):

    storage_client = storage.Client()

    blobs = storage_client.list_blobs(bucket_name, prefix=prefix, delimiter=delimiter)

    #print("Blobs:")
    for blob in blobs:
      #print(blob.name)
      video_list.append(blob.name)




In [ ]:
"""if __name__ == "__main__":
    list_blobs_with_prefix(
        bucket_name="video_list_capstoneyt_2022", prefix="videos/", delimiter="/"
    )
"""

'if __name__ == "__main__":\n    list_blobs_with_prefix(\n        bucket_name="video_list_capstoneyt_2022", prefix="videos/", delimiter="/"\n    )\n'

In [ ]:
list_blobs_with_prefix(
        bucket_name="videos_final", prefix="", delimiter="/"
    )

In [ ]:
len(video_list)

1083

In [ ]:
def list_blobs_with_prefix1(bucket_name, prefix, out_list, delimiter=None):
    storage_client = storage.Client()
    blobs = storage_client.list_blobs(bucket_name, prefix=prefix, delimiter=delimiter)

    #print("Blobs:")
    for blob in blobs:
      #print(blob.name)
      out_list.append(blob.name)

In [ ]:
storage_client = storage.Client()
bucket = storage_client.get_bucket('video_list_capstoneyt_2022')

outputSpeech=[]
list_blobs_with_prefix1(bucket_name="video_list_capstoneyt_2022", prefix="outputSpeech/", delimiter="/",out_list=outputSpeech)

outputDetection=[]      
list_blobs_with_prefix1(bucket_name="video_list_capstoneyt_2022", prefix="outputDetection/", delimiter="/",out_list=outputDetection)

outputOCR=[]
list_blobs_with_prefix1(bucket_name="video_list_capstoneyt_2022", prefix="outputOCR/", delimiter="/",out_list=outputOCR)

In [ ]:
"""
from google.colab import auth
auth.authenticate_user()
GCP_PROJECT_ID = 'stoked-dominion-341317'
PROJECT_NUMBER = '725342657078'
project_id = 'nifty-depth-246308'
!gcloud config set project {GCP_PROJECT_ID}
!gsutil ls
"""

"\nfrom google.colab import auth\nauth.authenticate_user()\nGCP_PROJECT_ID = 'stoked-dominion-341317'\nPROJECT_NUMBER = '725342657078'\nproject_id = 'nifty-depth-246308'\n!gcloud config set project {GCP_PROJECT_ID}\n!gsutil ls\n"

In [ ]:
#!gsutil -m cp -r gs://video_list_capstoneyt_2022/videos /content/gdrive/MyDrive/"Systems '22"/videos

In [ ]:
#!mv videos/videos/*.mp4 videos/
#!mv videos/videos_latest/videos/*.mp4 videos/

In [ ]:
outputVid=[]
list_blobs_with_prefix1(bucket_name="video_list_capstoneyt_2022", prefix="videos_final/", delimiter="/",out_list=outputVid)
print(outputVid)

[]


### **Features Extraction**

1. speech transcription (optional) 

2. shot change detection

3. object detection

4. OCR text detection

In [ ]:

def process(video_list,option='all'):
  for video in video_list:
    temp = video.split('/')
    id=temp[-1][:-4]
    #if option=='Speech' and 'outputSpeech/'+id+'.json' in outputSpeech:
    #  print('%s already in Speech folder.'%id)
    #  continue
    if option=='Detection' and 'outputDetection/'+id+'.json' in outputDetection:
      print('%s already in Detection folder.'%id)
      continue
    elif option=='OCR' and 'outputOCR/'+id+'.json' in outputOCR:
      print('%s already in OCR folder.'%id)
      continue
    else:
      #gs_URI = 'gs://video_list_capstoneyt_2022/'+ video
      gs_URI = 'gs://videos_final/'+ video
      print("here", gs_URI)
      time.sleep(1)

      #the features 
      features = [videointelligence.Feature.SPEECH_TRANSCRIPTION,videointelligence.Feature.SHOT_CHANGE_DETECTION, videointelligence.Feature.OBJECT_TRACKING, videointelligence.Feature.TEXT_DETECTION]
      
      #configuration
      #config = videointelligence.SpeechTranscriptionConfig(language_code="en-US", enable_automatic_punctuation=True, enable_word_confidence=False)
      config1 = videointelligence.ShotChangeDetectionConfig()
      config2 = videointelligence.ObjectTrackingConfig()
      config3 = videointelligence.TextDetectionConfig()

      #video context
      #if option=='Speech':
      #  video_context = videointelligence.VideoContext(speech_transcription_config=config)
      #  features=features[:1]
      if option=='Detection':
        video_context = videointelligence.VideoContext(shot_change_detection_config=config1, object_tracking_config=config2)
        features=features[1:3]
      elif option=='OCR':
        video_context = videointelligence.VideoContext(text_detection_config=config3)
        features=features[3:]
      else:
        print('Invalid feature name!')
      #else:
      #  video_context = videointelligence.VideoContext(speech_transcription_config=config,shot_change_detection_config=config1, object_tracking_config=config2,text_detection_config=config3)

      operation = video_client.annotate_video(
        request={
            "features": features,
            "input_uri": gs_URI,
            "video_context": video_context,
            "output_uri": 'gs://video_list_capstoneyt_2022/output'+option+'/'+ id +'.json'
            #"output_uri": 'gs://videos_final/output'+option+'/'+ id +'.json'
        }
      )

      print("Processing video for %s."%option.lower())

      result = operation.result()
      annotation_results = result.annotation_results[0]

      print("Completed\n")
      

Multithreading videoprocessing

In [ ]:
#only include Detection and OCR. We will use Cloud speech to process trascripts.
features=['Detection','OCR']

def processing(threadName,START,end,op_list=features):
  start=START
  print('start: %d    end: %d'%(start,end))
  while start+5<=end:
    print("%s: %s" % (threadName, time.ctime(time.time())))
    print('video %d to %d'%(start,start+4))
    temp_list=video_list[start:start+5]
    for op in op_list:
      process(temp_list,option=op)
    start+=5

  temp_list=video_list[start:end]
  print("%s: %s" % (threadName, time.ctime(time.time())))
  print('video %d to %d'%(start,end))
  for op in op_list:
      process(temp_list,option=op)

In [ ]:
print(len(video_list))
print(len(list(set(video_list))))

1083
1083


In [ ]:

"""
DONT'T RUN option='speech', there's a separate script for transcript.

MIN is the starting index, and MAX is the ending index in the processing.
step is the number of videos processing in each thread (optimal range is 20~40).

When at this code section, click "execute->run before".

before running this section, modify MIN and MAX
"""

import threading
import time

MIN=0
MAX=len(video_list)
step=40

class myThread(threading.Thread):
   def __init__(self, threadID, name, counter,begin,end):
      threading.Thread.__init__(self)
      self.threadID = threadID
      self.name = name
      self.counter=counter
      self.begin=begin
      self.end=end
   def run(self):
      print("Starting " + self.name)
      processing(self.name,self.begin,self.end)

threads = []
threadLock = threading.Lock()

# Create new threads
# Add threads to thread list
begin=MIN
end=begin+step
j=1
while begin<MAX: 
  if end<=MAX:
    cur_thread = myThread(j, "Thread-%d"%j, 1, begin,end)
  else:
    cur_thread = myThread(j, "Thread-%d"%j, 1,begin,MAX)
  begin=end
  end=begin+step
  j+=1
  threads.append(cur_thread)

# Start new Threads
for t in threads:
    t.start()

# Wait for all threads to complete
for t in threads:
    t.join()
print("Exiting Main Thread")

Starting Thread-1
start: 0    end: 400
Thread-1: Thu Apr 21 17:16:03 2022
video 0 to 4
here gs://videos_final/-5yWbp6B-eA.mp4
Starting Thread-2
start: 400    end: 800
Thread-2: Thu Apr 21 17:16:03 2022
video 400 to 404
MI4J7Uo4A7s already in Detection folder.
MKS2Hm5ofBE already in Detection folder.
MPfBGjP-cQs already in Detection folder.
MThqK9flPj0 already in Detection folder.
MUWG_OGtg2Q already in Detection folder.
MI4J7Uo4A7s already in OCR folder.
MKS2Hm5ofBE already in OCR folder.
MPfBGjP-cQs already in OCR folder.
MThqK9flPj0 already in OCR folder.
MUWG_OGtg2Q already in OCR folder.
Thread-2: Thu Apr 21 17:16:03 2022
video 405 to 409
M_zxqgRFTOU already in Detection folder.
MgNMqoiJLrw already in Detection folder.
MiK_sS-ckbU already in Detection folder.
Mko250V6PYI already in Detection folder.
Mlx_eXtHRgQ already in Detection folder.
M_zxqgRFTOU already in OCR folder.
MgNMqoiJLrw already in OCR folder.
MiK_sS-ckbU already in OCR folder.
Mko250V6PYI already in OCR folder.
Mlx_

### Debug speech





In [ ]:
from google.cloud import speech

%ls ./text/transcript_text/unlabeled/
AUDIO_PATH = "./text/audios-wav"
SAVE_PATH = "./text/transcript_text/unlabeled/raw_text/"
AUDIO_ID_LIST = os.listdir(AUDIO_PATH)

clean_text/  raw_text/


In [ ]:
def sample_long_running_recognize(storage_uri, filename):
    """
    Transcribe long audio file from Cloud Storage using asynchronous speech
    recognition
    Args:
      storage_uri URI for audio file in Cloud Storage, e.g. gs://[BUCKET]/[FILE]
    """

    client = speech.SpeechClient()

    # storage_uri = 'gs://cloud-samples-data/speech/brooklyn_bridge.raw'

    # Sample rate in Hertz of the audio data sent
    sample_rate_hertz = 16000

    # The language of the supplied audio
    language_code = "en-US"

    # Encoding of audio data sent. This sample sets this explicitly.
    # This field is optional for FLAC and WAV audio formats.
    encoding = speech.RecognitionConfig.AudioEncoding.LINEAR16
    config = {
        "audio_channel_count": 2,
        "enable_separate_recognition_per_channel": False,
        # "sample_rate_hertz": sample_rate_hertz,
        "language_code": language_code,
        "enable_automatic_punctuation": True,
        "encoding": encoding,
    }
    audio = {"uri": storage_uri}

    operation = client.long_running_recognize(request={"config": config, "audio": audio})

    print(u"Waiting for operation to complete...")
    response = operation.result()

    filename = filename.split('.')[0].split('/')[-1]

    transcript_json = []
    transcript_str = ''
    for idx, result in enumerate(response.results):
        # First alternative is the most probable result
        alternative = result.alternatives[0]
        print(u"Transcript: {}".format(alternative.transcript))
        print(u"Confidence: {}".format(alternative.confidence))
        transcript_str += alternative.transcript
        transcript_json.append({"transcript":alternative.transcript, "confidence":alternative.confidence})
    with open(os.path.join(SAVE_PATH, filename+'.txt'), 'w') as f1:
        f1.write(transcript_str)
    with open(os.path.join(SAVE_PATH, filename+'.json'), 'w') as f2:
        json.dump(transcript_json, f2)

In [ ]:
def get_processed_filename_set():
    processed_filename_set = os.listdir(SAVE_PATH)
    processed_filename_set = [i.split('.')[0] for i in processed_filename_set]
    return processed_filename_set

In [ ]:
def get_exist_filename_set(bucket_name):

    client = storage.Client()
    # bucket = storage.Bucket("fyanvideos")
    blob_list = client.list_blobs(bucket_name)
    filenames = set()
    for blob in blob_list:
        filenames.add(blob.name)
        # print(blob.name)
    return filenames

In [ ]:
BUCKET_NAME ="audio_wav_list_capstoneyt_2022"
def main():
    filenames = get_exist_filename_set(BUCKET_NAME)
    processed_filename_set = get_processed_filename_set()
    print(processed_filename_set)
    processed=0
    
    for filename in list(filenames)[:350]:
        pre_name = filename.split('.')[0].split('/')[-1]
        if pre_name not in processed_filename_set:
            if processed>0:
              print('already processed %d'%processed)
              processed=0
            storage_uri = 'gs://{}/{}'.format(BUCKET_NAME, filename)
            sample_long_running_recognize(storage_uri, filename)  
        else:
            processed+=1
    print('already processed %d'%processed)

In [ ]:
filenames = get_exist_filename_set(BUCKET_NAME)
filenames=[f.split('.')[0].split('/')[-1] for f in filenames]
processed_filename_set = get_processed_filename_set()

f=set(filenames)-set(processed_filename_set)

print(len(list(f)))
main()

154
